# Runs 

<div style="text-align:center">
    <img src="../../images/mlflow_run.jpeg" alt="MLFlow Run" />
</div>


A MLflow run is a unit of work in MLflow that represents the execution of a machine learning experiment or a piece of code. It tracks the parameters, metrics, artifacts, and metadata associated with the run. MLflow runs allow you to log and track experiments, compare different runs, and reproduce results. Each run is associated with an experiment and can have multiple tags, parameters, metrics, and artifacts.

## Creating a MLflow Run

### Using start_run

```python
mlflow.start_run()
```

In [ ]:
import mlflow 
from mlflow_for_ml_dev.experiments.exp_utils import get_or_create_experiment

In [ ]:
experiment_name = "creating_runs"
tags = {"project_name":"UNDEFINED", "topic":"run_management"}
experiment = get_or_create_experiment(experiment_name=experiment_name, tags=tags)

In [ ]:
run_tags = {"tag1": "value1", "tag2": "value2"}
run = mlflow.start_run(
    run_name = "Run 1",
    experiment_id = experiment.experiment_id,
    description="This is a run description",
    tags= run_tags
)

In [ ]:
run.to_dictionary()

In [ ]:
active_run = mlflow.active_run()
print(type(active_run))

In [ ]:
mlflow.end_run()

#### Using the context Manager


> About `mlflow.start_run()`
> 
> The return value of `mlflow.start_run()` can be used as a context manager within a `with` block. Otherwise, you must call `end_run()` to terminate the current run.

Example:

```python
with mlflow.start_run() as run:
    print("Log metrics and params")
```

In [ ]:
mlflow.end_run() # end the active run to start a new one
with mlflow.start_run(run_name="Run 2", experiment_id=experiment.experiment_id) as run:
    active_run = mlflow.active_run()
    print(type(active_run))  
    print("Active Run: ", run.info.run_id)
    print("\n \n")



active_run = mlflow.active_run()
print(type(active_run))    

## Using MlflowClient

In [ ]:
client = mlflow.MlflowClient()

In [ ]:
created_run = client.create_run(experiment_id=experiment.experiment_id, run_name="test_run", tags={"tag1": "value1", "tag2": "value2"})

In [ ]:
run = mlflow.active_run()
type(run)

## Updating Run Tags

In [ ]:
with mlflow.start_run(run_name="Run 3", experiment_id=experiment.experiment_id) as run:
    mlflow.set_tag("tag3", "value3")
    
    # Set tags
    mlflow.set_tags({"tag4": "value4", "tag5": "value5"})

## Update run description

In [ ]:
with mlflow.start_run(run_name="Run 4", experiment_id=experiment.experiment_id) as run:
    #Update description
    mlflow.set_tag("mlflow.note.content", "This is a new description")

## Retrieve run information

In [ ]:
run = mlflow.get_run(run.info.run_id)

In [ ]:
run.info.run_id

In [ ]:
run.data.tags